In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [2]:
def disease_labels(line):
    if 'positive' in line:
        dis = 1
    else:
        dis = 0
            
    return dis
        

def get_patient_id(line):
    loc = line.find('patient')
    #print(loc)
    
    pid = int(line[loc+7:loc+12])
    #print(pid)
    
    return pid

def get_view(line):
    if 'SHOULDER' in line:
        view = 0
    elif 'HUMERUS' in line:
        view = 1
    elif 'FINGER' in line:
        view = 2
    elif 'ELBOW' in line:
        view = 3
    elif 'WRIST' in line:
        view = 4
    elif 'FOREARM' in line:
        view = 5
    elif 'HAND' in line:
        view = 6
    else:
        view = -1
        
    return view

In [3]:
filename = []
patient_id = []
sex = []
age = []
extract = 'Frontal'
view = []
dis = []

view_labels = ['SHOULDER', 'HUMERUS', 'FINGER', 'ELBOW', 'WRIST', 'FOREARM', 'HAND']

i = 0

with open('MURA-v1.1/train_image_paths.csv', 'r') as f:
    
    
    for line in f:
        line = line.replace('\n','')
        #line = line.split(',')
        #print(line)
        
        filename.append(line)
        patient_id.append(get_patient_id(line))
        dis.append(disease_labels(line))
        i = i+1
        
        v = get_view(line)
        if v==-1:
            print(line)
            break
        else:
            view.append(v)
        #print(dis)
        
        #break

diseases = np.array(dis)
view = np.array(view)


print(diseases.shape)
print('Total no of images: ', i)

(36808,)
Total no of images:  36808


In [ ]:
import torch
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True).to(device)
# or any of these variants
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet169', pretrained=True).to(device)
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet201', pretrained=True).to(device)
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet161', pretrained=True).to(device)
model.eval()


def run_through_denseNetPyTorch(img, densenet_model, device):
    
    img = torch.unsqueeze(img.to(device), 0)
    
    with torch.no_grad():
        features = densenet_model.features( img )
        f_relu = torch.nn.functional.relu( features )
        pool = torch.nn.functional.adaptive_avg_pool2d( f_relu, (1,1) )

        y_st = pool.cpu().numpy().reshape(-1)
    
    return y_st

In [ ]:
from imageio import imread 
import PIL 
from PIL import Image 
from torchvision import transforms 

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


images = []

X_train_0 = []

for i in range(len(filename)):
    if i%10000 == 0:
        print('Completed ', i, ' of ', len(filename))
    
    
    fname = filename[i]
    #print(fname)
    img = imread(fname, as_gray=False, pilmode="RGB") 
    
    img = Image.fromarray(img)
    img = PIL.ImageOps.equalize(img)
    img = preprocess(img)
    
    X_train_0.append(run_through_denseNetPyTorch(img=img, densenet_model=model, device=device))
    
    #images.append(img)

In [ ]:
data_dict = {}
data_dict['filename'] = filename
data_dict['patient_id'] = patient_id
data_dict['view'] = view
data_dict['view_label'] = view_labels

X_train_0 = np.array(X_train_0)

with open("MURA_Train_DenseNet121_Complete.npy", 'wb') as f:
    np.save(f,X_train_0)
    np.save(f,diseases)
    np.save(f,data_dict)

In [ ]:
from sklearn.model_selection import train_test_split 

X_train = X_train_0
y_train = diseases


In [ ]:
import umap 
 
    
import matplotlib.pyplot as plt
%matplotlib inline
    
np.random.seed(100) 
ump = umap.UMAP(n_neighbors=10, 
        min_dist=0.001, 
        n_components=2, 
        random_state=150, 
        n_epochs = 300,
        metric= 'euclidean') 
 
y_umap = ump.fit_transform(X_train) 
 
plt.figure() 
plt.scatter(y_umap[:,0], y_umap[:,1], c=y_train, s=0.01, cmap='Spectral') 
 
cbar = plt.colorbar(boundaries=np.arange(3)-0.5) 
cbar.set_ticks(np.arange(3)) 

#cbar.set_ticklabels(classes) 

In [ ]:
%matplotlib notebook
plt.figure() 
plt.scatter(y_umap[:,0], y_umap[:,1], c=y_train, s=0.01, cmap='Spectral') 
 
cbar = plt.colorbar(boundaries=np.arange(3)-0.5) 
cbar.set_ticks(np.arange(2)) 

In [ ]:
%matplotlib notebook
plt.figure() 
plt.scatter(y_umap[:,0], y_umap[:,1], c=view, s=0.01, cmap='Spectral') 
 
cbar = plt.colorbar(boundaries=np.arange(len(view)+1)-0.5) 
cbar.set_ticks(np.arange(len(view))) 